In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from typing import List
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import time
import json

In [2]:
toner_url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=10000010002'

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

prod_dict = {
    '상품 카테고리': '립메이크업',
    '정보': []
}

# Chrome 설정
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument(f"user-agent={user_agent}")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    driver.get(toner_url)
    page_count = 1
    
    while True:
        print(f'=== {page_count} 페이지 스크래핑 시작 ===')
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'tx_name'))
        )
        total_products = len(driver.find_elements(By.CLASS_NAME, 'tx_name'))
        print(f'{total_products}개 제품')
        
        for i in range(1, total_products + 1):
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'tx_name'))
                )
                
                products = driver.find_elements(By.CLASS_NAME, 'tx_name')
                prices = driver.find_elements(By.CLASS_NAME, 'tx_cur')
                
                product_nm = products[i - 1].text
                product_price = prices[i - 1].text
                
                WebDriverWait(products[i - 1], 15).until(
                    EC.element_to_be_clickable((By.XPATH, '..'))
                )
                a_click = products[i - 1].find_element(By.XPATH, '..')
                a_click.click()
                time.sleep(5)
                ingredient_text = '성분 정보 없음'
                
                try:
                    WebDriverWait(driver, 15).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[@id="buyInfo"]'))
                    )
                    buy_info = driver.find_element(By.XPATH, '//*[@id="buyInfo"]')
                    buy_info.click()
                    time.sleep(5)
                    WebDriverWait(driver, 15).until(
                        EC.presence_of_all_elements_located((By.TAG_NAME, 'dt'))
                    )
                    dts = driver.find_elements(By.TAG_NAME, 'dt')
                    
                    for dt in dts:
                        if '성분' in dt.text:
                            dd = dt.find_element(By.XPATH, 'following-sibling::dd')
                            ingredient_text = dd.text.strip()
                            break
                except Exception as e:
                    print(f'"{product_nm}" 성분 정보 오류: {e}')
                    
                prod_dict['정보'].append({
                    '제품명': product_nm,
                    '가격': product_price,
                    '성분': ingredient_text
                })
                driver.back()
                time.sleep(5)
            except Exception as e:
                print(f'전체 페이지 스크래핑 오류: {e}')
                import traceback
                traceback.print_exc()
                
        try:
            WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="Container"]/div[2]/strong/following-sibling::a[1]'))
            )
            next_page_button = driver.find_element(By.XPATH, '//*[@id="Container"]/div[2]/strong/following-sibling::a[1]')
            print(f'=== {page_count} 페이지 스크래핑 완려, "{next_page_button.text}페이지로 넘어감"')
            next_page_button.click()
            page_count += 1
            time.sleep(5)
        except Exception as e:
            print(f'페이지 끝 {page_count} 스크래핑 종료')
            break
except Exception as e:
    print(f'드라이버 오류 {e}')
    
finally:
    driver.quit()
    
    with open('립메이크업.json', 'w') as f:
        json.dump(prod_dict, f, indent=4, ensure_ascii=False)

=== 1 페이지 스크래핑 시작 ===
24개 제품
"[11월 올영픽/주토피아에디션] 토니모리 겟잇 틴트 워터풀 버터" 성분 정보 오류: Message: 
Stacktrace:
#0 0x5d57cddc0fea <unknown>
#1 0x5d57cd83ed06 <unknown>
#2 0x5d57cd890e6d <unknown>
#3 0x5d57cd891101 <unknown>
#4 0x5d57cd8df6b4 <unknown>
#5 0x5d57cd8b6e3d <unknown>
#6 0x5d57cd8dcb3b <unknown>
#7 0x5d57cd8b6be3 <unknown>
#8 0x5d57cd8832d2 <unknown>
#9 0x5d57cd883f91 <unknown>
#10 0x5d57cdd849e8 <unknown>
#11 0x5d57cdd8884f <unknown>
#12 0x5d57cdd6bec9 <unknown>
#13 0x5d57cdd893f5 <unknown>
#14 0x5d57cdd5174f <unknown>
#15 0x5d57cddadcc8 <unknown>
#16 0x5d57cddadea3 <unknown>
#17 0x5d57cddbff83 <unknown>
#18 0x7fa6b129caa4 <unknown>
#19 0x7fa6b1329c6c <unknown>

=== 1 페이지 스크래핑 완려, "2페이지로 넘어감"
=== 2 페이지 스크래핑 시작 ===
24개 제품
=== 2 페이지 스크래핑 완려, "3페이지로 넘어감"
=== 3 페이지 스크래핑 시작 ===
24개 제품
=== 3 페이지 스크래핑 완려, "4페이지로 넘어감"
=== 4 페이지 스크래핑 시작 ===
24개 제품
=== 4 페이지 스크래핑 완려, "5페이지로 넘어감"
=== 5 페이지 스크래핑 시작 ===
24개 제품
=== 5 페이지 스크래핑 완려, "6페이지로 넘어감"
=== 6 페이지 스크래핑 시작 ===
24개 제품
=== 6 페이지 스크래핑 완려, "7페이지로 넘어감"

KeyboardInterrupt: 

In [25]:
prod_dict['정보']

[{'제품명': '[11월 올영픽] 바이오더마 하이드라비오 토너 500ml 기획(+화장솜 20매 증정)',
  '가격': '26,900원',
  '성분': '정제수, 글리세린, 폴리솔베이트20, 다이소듐이디티에이, 세트리모늄브로마이드, 향료, 나이아신아마이드, 자일리톨, 알란토인, 프룩토올리고사카라이드, 만니톨, 헥실데칸올, 소듐하이드록사이드, 시트릭애씨드, 람노오스, 사과씨추출물, 유채스테롤, 토코페롤'},
 {'제품명': '[증량기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추가 증정)',
  '가격': '14,920원',
  '성분': '정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인, 알란토인, 판테놀, 쇠비름추출물, 트레할로오스, 소듐하이알루로네이트, 하이드롤라이즈드하이알루로닉애씨드(100ppm), 소듐하이알루로네이트크로스폴리머, 하이드롤라이즈드소듐하이알루로네이트, 소듐아세틸레이티드하이알루로네이트, 2,3-부탄다이올, 펜틸렌글라이콜, 말라카이트추출물, 다이소듐이디티에이, 에틸헥실글리세린'},
 {'제품명': '[대용량 기획] 아누아 어성초 77 수딩 토너 350ml 기획 (+350ml 리필팩)',
  '가격': '27,500원',
  '성분': '약모밀꽃/잎/줄기수(77%), 정제수, 1,2-헥산다이올, 글리세린, 베타인, 부틸렌글라이콜, 아이소펜틸다이올, 판테놀, 아크릴레이트/C10-30알킬아크릴레이트크로스폴리머, 트로메타민, 병풀추출물, 우엉뿌리추출물, 진흙버섯추출물, 쇠비름추출물, 마트리카리아꽃추출물, 체이스트트리추출물, 소듐하이알루로네이트, 하이드록시프로필트라이모늄하이알루로네이트, 소듐아세틸레이티드하이알루로네이트, 하이드롤라이즈드하이알루로닉애씨드, 하이알루로닉애씨드, 소듐하이알루로네이트크로스폴리머, 포타슘하이알루로네이트, 다이소듐이디티에이'},
 {'제품명': '[미스트 기획/화잘먹] 라네즈 크림스킨 170ml 리필기획 (+170ml 리필+50ml+미스트펌프)',
  '가격': '

In [29]:
with open('토너.json', 'w', encoding='utf-8') as f:
    json.dump(prod_dict, f, indent=4, ensure_ascii=False)